# Implementation of Langchain-OpenAI to build AI chatbots

### Using Few-shot templates, prompts with examples

Refer [here](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) for information about Prompt Templates.

Simple, prompt templates are prompt recipes of prompts which can be used with multiple models. We'll se that with examples.

### Loading model

In [2]:
# Import the required modules
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.schema import AIMessage, SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI

In [3]:
# load .env file
load_dotenv(find_dotenv())

True

In [4]:
# get open ai key and config

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

### setting up template

In [5]:
# examples for template
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

In [6]:
# create a example template
example_template = """
User: {query}
AI: {answer}
"""

In [7]:
# promplt template from above info
example_prompt = PromptTemplate(
    template=example_template,
    input_variables = ["query", "answer"]
)

In [8]:
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [9]:
# create few shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n####\n"
)

In [10]:
query = "What is the meaning of life?"

print(few_shot_prompt.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

####

User: How are you?
AI: I can't complain but sometimes I still do.

####

User: What time is it?
AI: It's time to get a watch.

####

User: What is the meaning of life?
AI: 


In [31]:
# test a question
chat_model = ChatOpenAI()

chat_model([HumanMessage(content=few_shot_prompt.format(query='What should i do now?'))]).content

"You should probably ask someone who knows what they're doing."

If there is a large list of examples then we can use length baseed example selector.

In [32]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [33]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

In [34]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

The length based example selector will vary the number of examples selected based on the length of the query. This is helpful in limiting the tokens.


Shorter query, more examples. Longer query, less examples.

In [35]:
print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI: 


In [36]:
chat_model([HumanMessage(content=dynamic_prompt_template.format(query='How do birds fly?'))]).content

'With their wings, of course. Or maybe they just believe in themselves really hard.'

In [37]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


In [38]:
chat_model([HumanMessage(content=dynamic_prompt_template.format(query=query))]).content

"Well, you could try using a phone. Or maybe a carrier pigeon, if you're feeling old school."